In [30]:
#descargar archivos desde url y almacenarlos en carpeta local
import urllib.request as request
import csv
r = request.urlopen('https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/drivers/truck_event_text_partition.csv').read().decode('utf8').split("\n")
reader = csv.reader(r)
#para imprimir el archivo
''' 
for line in reader:
    print(line)    
'''


' \nfor line in reader:\n    print(line)    \n'

In [32]:
#otra manera e importar los archgivos desde internet
df = pd.read_csv('https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/drivers/truck_event_text_partition.csv')
df.to_csv('truck_event_text_partition.csv')

In [28]:
import os

for root, dirs, files in os.walk('/TEMP/'):
    for file in files:
        if file.endswith('.ipynb'):
            print(os.path.join(root, file))


In [17]:
#prueba de commit en github con el archivo de jupyter lab
filenames = [
    "drivers.csv",
    "timesheet.csv",
    "truck_event_text_partition.csv",
]
!ls -1 /tmp

"ls" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [6]:
#Descargando datos desde repo github
import pandas as pd

url = "https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/drivers/drivers.csv"
drivers = pd.read_csv(url)
drivers.head()

,driverId,name,ssn,location,certified,wage-plan
0,10,George Vetticaden,621011971,244-4532 Nulla Rd.,N,miles
1,11,Jamie Engesser,262112338,366-4125 Ac Street,N,miles
2,12,Paul Coddin,198041975,Ap #622-957 Risus. Street,Y,hours
3,13,Joe Niemiec,139907145,2071 Hendrerit. Ave,Y,hours
4,14,Adis Cesir,820812209,Ap #810-1228 In St.,Y,hours


In [25]:
import pandas as pd
url2="https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/drivers/truck_event_text_partition.csv"
truck_event_text_partition=pd.read_csv(url2)
truck_event_text_partition.head()

,driverId,truckId,eventTime,eventType,longitude,latitude,eventKey,CorrelationId,driverName,routeId,routeName,eventDate
0,14,25,59:21.4,Normal,-94.58,37.03,14|25|9223370572464814373,3.660000e+18,Adis Cesir,160405074,Joplin to Kansas City Route 2,2016-05-27-22
1,18,16,59:21.7,Normal,-89.66,39.78,18|16|9223370572464814089,3.660000e+18,Grant Liu,1565885487,Springfield to KC Via Hanibal,2016-05-27-22
2,27,105,59:21.7,Normal,-90.21,38.65,27|105|9223370572464814070,3.660000e+18,Mark Lochbihler,1325562373,Springfield to KC Via Columbia Route 2,2016-05-27-22
3,11,74,59:21.7,Normal,-90.20,38.65,11|74|9223370572464814123,3.660000e+18,Jamie Engesser,1567254452,Saint Louis to Memphis Route2,2016-05-27-22
4,22,87,59:21.7,Normal,-90.04,35.19,22|87|9223370572464814101,3.660000e+18,Nadeem Asghar,1198242881,Saint Louis to Chicago Route2,2016-05-27-22


Preparacion

In [9]:
import sqlite3
conn=sqlite3.connect(':memory:')## aca se indica el nombre de la db
cur= conn.cursor()

Carga de los datos de los eventos de los conductores

In [19]:
#
# executescript() permite enviar varios comandos de SQL
# en la misma cadena de texto.
#

conn.executescript(
    """
DROP TABLE IF EXISTS truck_events;

CREATE TABLE truck_events (driverId       INT,
                           truckId        INT,
                           eventTime      STRING,
                           eventType      STRING,
                           longitude      DOUBLE,
                           latitude       DOUBLE,
                           eventKey       STRING,
                           correlationId  STRING,
                           driverName     STRING,
                           routeId        STRING,
                           routeName      STRING,
                           eventDate      STRING);
"""
)
conn.commit()

In [20]:
#
# La función execute() permite enviar únicamente un comando SQL
# al motor de la base de datos. El siguiente comando es equivalente
# a `SHOW TABLES;` en SQL
#
cur.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

[('truck_events',)]

In [21]:
#
# La siguiente sentencia devuelve un string que
# representa los comandos sql para crear la tabla
#
cur.execute(
    "SELECT sql FROM sqlite_master WHERE type='table' and name='truck_events';"
).fetchall()

[('CREATE TABLE truck_events (driverId       INT,\n                           truckId        INT,\n                           eventTime      STRING,\n                           eventType      STRING,\n                           longitude      DOUBLE,\n                           latitude       DOUBLE,\n                           eventKey       STRING,\n                           correlationId  STRING,\n                           driverName     STRING,\n                           routeId        STRING,\n                           routeName      STRING,\n                           eventDate      STRING)',)]

In [22]:
#
# Se imprime el comando sql equivalente para cada
# tabla existente en la base de datos
#
for a in cur.execute("SELECT sql FROM sqlite_master WHERE type='table';").fetchall():
    print(a[0])

CREATE TABLE truck_events (driverId       INT,
                           truckId        INT,
                           eventTime      STRING,
                           eventType      STRING,
                           longitude      DOUBLE,
                           latitude       DOUBLE,
                           eventKey       STRING,
                           correlationId  STRING,
                           driverName     STRING,
                           routeId        STRING,
                           routeName      STRING,
                           eventDate      STRING)


<h4>Carga de datos usando INSERT INTO</h4>
Este método de carga consiste en crear una lista de tupas, donde cada elemento de la tupla es un campo de la tabla.


In [35]:
with open('truck_event_text_partition.csv','rt') as f:
    data=f.readlines()
data[:3]

[',driverId,truckId,eventTime,eventType,longitude,latitude,eventKey,CorrelationId,driverName,routeId,routeName,eventDate\n',
 '0,14,25,59:21.4,Normal,-94.58,37.03,14|25|9223370572464814373,3.66e+18,Adis Cesir,160405074,Joplin to Kansas City Route 2,2016-05-27-22\n',
 '1,18,16,59:21.7,Normal,-89.66,39.78,18|16|9223370572464814089,3.66e+18,Grant Liu,1565885487,Springfield to KC Via Hanibal,2016-05-27-22\n']